In [1]:
!pip install dash

  Using cached werkzeug-3.0.2-py3-none-any.whl.metadata (4.1 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
  Using cached MarkupSafe-2.1.5-cp311-cp311-win_amd64.whl.metadata (3.1 kB)
   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.2 MB 6.1 MB/s eta 0:00:02
   -- ------------------------------------- 0.5/10.2 MB 6.8 MB/s eta 0:00:02
   --- ------------------------------------ 0.9/10.2 MB 8.2 MB/s eta 0:00:02
   ---- ----------------------------------- 1.1/10.2 MB 7.2 MB/s eta 0:00:02
   ------ --------------------------------- 1.6/10.2 MB 7.9 MB/s eta 0:00:02
   ------- -------------------------------- 2.0/10.2 MB 7.8 MB/s eta 0:00:02
   --------- ------------------------------ 2.3/10.2 MB 7.7 MB/s eta 0:00:02
   ---------- ----------------------------- 2.7/10.2 MB 7.4 MB/s eta 0:00:02
   ------------ --------------------------- 3.2/10.2 MB 7.7 MB/s eta 0:00:01
   --------------- --------------

In [23]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import time


In [32]:
# Create a dataframe with columns 'Date', 'High', 'Low', 'Open', 'Close'
df = pd.DataFrame({
    'Date': ['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04', '2021-01-05'],
    'High': [110, 120, 130, 140, 150],
    'Low': [90, 100, 110, 120, 130],
    'Open': [95, 105, 115, 125, 145],
    'Close': [105, 115, 125, 135, 135]
})
df.Date = pd.to_datetime(df.Date)
df.dtypes

Date     datetime64[ns]
High              int64
Low               int64
Open              int64
Close             int64
dtype: object

In [33]:
app = Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='candlestick-chart'),
    dcc.Interval(id='interval', interval=1000)
])

@app.callback(
    Output(component_id='candlestick-chart', component_property='figure'),
    Input(component_id='interval', component_property='n_intervals')
)
def update_candlestick_chart(n_intervals):
    fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                                         open=df['Open'],
                                         high=df['High'],
                                         low=df['Low'],
                                         close=df['Close'])])
    fig.update_layout(title='Candlestick Chart', xaxis_title='Date', yaxis_title='Price', xaxis_rangeslider_visible=False, height=600, width=1000)
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)
    rng = np.random.default_rng()
    while(True):
        ran_num = np.random.randint(100,150)
        # add new data to the dataframe
        new_data = pd.DataFrame({
            'Date': df.Date.tail(1) + pd.DateOffset(days=1),
            'High': [ran_num + rng.integers(10, 20)],
            'Low': [ran_num - rng.integers(10, 20)],
            'Open': [ran_num + rng.integers(-10, 10)],
            'Close': [ran_num + rng.integers(-10, 10)]
        })
        df = pd.concat([df, new_data], ignore_index=True)
        time.sleep(2)

KeyboardInterrupt: 